# Import Library 

In [1]:
import pandas as pd 
import numpy as np
from datetime import date

pd.set_option("display.max_columns",None)
pd.options.display.float_format = '{:.2f}'.format

# Import Dataset 

In [2]:
cycle = pd.read_csv(r"\Belajar\Jala Test\Dataset\cycles.csv")
farm = pd.read_csv(r"\Belajar\Jala Test\Dataset\farms.csv")
fasting = pd.read_csv(r"\Belajar\Jala Test\Dataset\fasting.csv")
fastings = pd.read_csv(r"\Belajar\Jala Test\Dataset\fastings.csv")
feed_tray = pd.read_csv(r"\Belajar\Jala Test\Dataset\feed_tray.csv")
feeds = pd.read_csv(r"\Belajar\Jala Test\Dataset\feeds.csv")
harvest = pd.read_csv(r"\Belajar\Jala Test\Dataset\harvests.csv")
measure = pd.read_csv(r"\Belajar\Jala Test\Dataset\measurements.csv")
mortal = pd.read_csv(r"\Belajar\Jala Test\Dataset\mortalities.csv")
ponds = pd.read_csv(r"\Belajar\Jala Test\Dataset\ponds.csv")
sampling = pd.read_csv(r"\Belajar\Jala Test\Dataset\samplings.csv")

# Data Understanding 

In [3]:
harvest.status.unique()

array(['Failed', 'Full', 'Partial', nan, 'FINISIHING',
       'Transfer ke N1 dan N2', 'Transfer ke 10', 'Transfer ke 9',
       'Transfer ke 8', 'finishing (pengesatan)', 'Parsial', 'Patial',
       'Transfer', 'Transfer ke Tambak'], dtype=object)

In [4]:
print(f"Harvest table - cycle id = {harvest.cycle_id.nunique()}")
print(f"Cycle tabel - cycle id = {cycle.id.nunique()}")

Harvest table - cycle id = 2490
Cycle tabel - cycle id = 2500


In [5]:
cycle.total_seed_type.unique()

array(['net', 'gross', 'actual', nan], dtype=object)

In [6]:
cycle.id.value_counts()

id
20124    4
20189    4
20190    4
24519    3
24304    3
        ..
16059    1
23342    1
26560    1
20981    1
20849    1
Name: count, Length: 2500, dtype: int64

In [7]:
(cycle.groupby('id')['total_seed'].nunique()).unique()

array([1], dtype=int64)

In [8]:
sampling['remark'].unique()[-10:]

array(['Berat : 982,3 (blm kurang air 1034)\nEkor : 84\nAbw : 11,69\nSize : 85,54\nAdg : 0,39',
       'berat total 461,7 gram; ekor 31; size 67,5',
       'gram;671 ekor;33  abw;20.3 size;49.2',
       'gram;1174 ekor;52 mbw;22.5 size!44.4', 'size 212', 'size 132',
       'berat : 316,35 gr ekor : 87', 'Sampling ke 2', 'size 293',
       'size 287'], dtype=object)

# Calculate Metrics

- Survival Rate

In [9]:
# Preprocess 
SR = cycle.copy()
SR.rename(columns={'id':'cycle_id'}, inplace=True)

# Join Data  
SR_join = pd.merge(SR.drop(['updated_at','created_at'],axis=1), harvest.drop(['updated_at','created_at'],axis=1), how='outer', on='cycle_id')

# Filter Data
panen = ['Full', 'Partial', 'FINISIHING','finishing (pengesatan)', 'Parsial', 'Patial'] # --> Asumsi bahwa status tidak gagal panen / pemindahan(transfer)

# Cleaning
SR = SR_join[SR_join['status'].isin(panen) & SR_join['status'].notna()].sort_values(by=['harvested_at','cycle_id'], ascending=True).reset_index(drop='first')
SR['started_at']=pd.to_datetime(SR['started_at'], format='mixed') 
SR['finished_at']=pd.to_datetime(SR['finished_at'], format='mixed') 

SR['long_cycle(day)'] = (SR['finished_at'] - SR['started_at']).dt.days

# Survival Rate % (End)
total_harvest = SR[['cycle_id','harvested_at','weight','size']].sort_values(by=['cycle_id','harvested_at'], ascending=True).reset_index(drop='first')
total_harvest['total_harvest(kg)'] = total_harvest['weight'] * total_harvest['size']

df_sr = pd.merge(total_harvest.groupby('cycle_id')[['total_harvest(kg)']].sum().reset_index(), SR[['cycle_id','total_seed']].drop_duplicates(), how='left', on='cycle_id')

df_sr['survival_rate(%)'] = round((df_sr['total_harvest(kg)']/df_sr['total_seed'])*100,2)


In [10]:
# Result
df_sr[df_sr['survival_rate(%)'] < 100][['cycle_id','survival_rate(%)']].describe()

,cycle_id,survival_rate(%)
count,1864.00,1864.00
mean,19696.42,58.47
std,5949.80,26.25
min,3458.00,0.00
25%,16045.75,37.47
50%,19686.50,61.75
75%,24769.25,80.73
max,29874.00,99.92


In [11]:
# Anomali 
df_sr[df_sr['survival_rate(%)'] > 100].describe()

,cycle_id,total_harvest(kg),total_seed,survival_rate(%)
count,476.00,476.00,476.00,476.00
mean,20028.16,419884.92,185927.85,1213.67
std,6245.35,2148621.58,148642.47,8665.74
min,4038.00,3127.00,10.00,100.03
25%,16337.00,78725.00,60000.00,106.73
50%,20799.00,192655.35,150000.00,115.52
75%,24886.50,347520.52,283096.00,137.60
max,29679.00,40000000.00,1200000.00,115817.39


**__Catatan__** : <p>
Berdasarkan analisa, terdapat banyak siklus yang memiliki total_seed dan harvest yang tidak sesuai, seperti total harvest > total_seed, sehingga menghasilkan survival_rate(%) end cultivation > 100%

In [12]:
# Preprocess Accumulative
id_list = total_harvest.cycle_id.unique().tolist()
total_harvest_acc = pd.DataFrame()

for id in id_list[:]:
    tabel = total_harvest[total_harvest['cycle_id'] == id]
    tabel['total_harvest_acc(kg)'] = tabel.groupby('cycle_id')['total_harvest(kg)'].cumsum()

    total_harvest_acc = pd.concat([total_harvest_acc, tabel], ignore_index=True)

df_sr_day = total_harvest_acc.copy()
df_sr_day = pd.merge(df_sr_day,SR[['cycle_id','total_seed']].drop_duplicates(), how='left',on='cycle_id')

# Survival Rate % (Pointed Day)
df_sr_day['survival_rate(%)'] = round((df_sr_day['total_harvest_acc(kg)']/df_sr_day['total_seed'])*100,2)

<ipython-input-12-d935f5022028>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tabel['total_harvest_acc(kg)'] = tabel.groupby('cycle_id')['total_harvest(kg)'].cumsum()
<ipython-input-12-d935f5022028>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tabel['total_harvest_acc(kg)'] = tabel.groupby('cycle_id')['total_harvest(kg)'].cumsum()
<ipython-input-12-d935f5022028>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

In [13]:
# Result
df_sr_day[df_sr_day['survival_rate(%)'] < 100][['cycle_id','survival_rate(%)']].describe()

,cycle_id,survival_rate(%)
count,7051.00,7051.00
mean,19858.13,43.90
std,5680.50,26.14
min,3458.00,0.00
25%,16343.50,21.56
50%,19688.00,40.00
75%,24691.00,64.28
max,29874.00,99.99


In [14]:
# Anomali
df_sr_day[df_sr_day['survival_rate(%)'] > 100].describe()

,cycle_id,weight,size,total_harvest(kg),total_harvest_acc(kg),total_seed,survival_rate(%)
count,959.00,959.00,959.00,959.00,959.00,959.00,959.00
mean,21100.06,1759.41,120.25,157067.37,508736.82,203689.31,1073.58
std,5337.97,11811.38,415.58,1494505.70,1977384.68,142356.45,7531.06
min,4038.00,0.50,19.00,37.30,3127.00,10.00,100.01
25%,18675.00,61.80,48.40,5604.70,125117.00,100000.00,107.11
50%,21596.00,350.00,80.00,30000.00,261209.18,175000.00,117.33
75%,24965.50,1270.50,125.00,97807.50,367283.07,286607.00,153.06
max,29679.00,250000.00,10000.00,40000000.00,40000000.00,1200000.00,115817.39


**__Catatan__** : <p>
Berdasarkan analisa, terdapat banyak siklus yang memiliki total_seed dan harvest yang tidak sesuai, seperti total harvest > total_seed, sehingga menghasilkan survival_rate(%) per day > 100%

- Average Daily Gain 

In [15]:
# Preprocess
sample = sampling.sort_values(by=['cycle_id','sampled_at']).reset_index(drop='first').drop(['updated_at','created_at','id','remark'],axis=1)
sample['sampled_at']=pd.to_datetime(sample['sampled_at'], format='mixed') 

In [16]:
# Count Distance Date

df_sample = pd.DataFrame()
id_list = sample.cycle_id.unique().tolist()

for id in id_list[:]:
    tabel = sample[sample['cycle_id'] == id].copy()
    
    tabel['distance_date'] = tabel['sampled_at'].diff().dt.days
    
    tabel['distance_date'] = tabel['distance_date'].fillna(0).astype(int)
    
    df_sample = pd.concat([df_sample, tabel], ignore_index=True)

# Calculate
adg = pd.DataFrame()
for id in id_list[:]:
    tabel = df_sample[df_sample['cycle_id'] == id].copy()
    
    tabel['average_daily_gain'] = tabel['average_weight'].diff()/tabel['distance_date']

    # Handling inf
    inf_mask = np.isinf(tabel['average_daily_gain'])
    tabel.loc[inf_mask, 'average_daily_gain'] = tabel['average_weight'].diff()[inf_mask]
    
    tabel['average_daily_gain'] = tabel['average_daily_gain'].fillna(0)
    
    adg = pd.concat([adg, tabel], ignore_index=True)


In [17]:
# Result
adg[['cycle_id','average_daily_gain']].describe()

,cycle_id,average_daily_gain
count,15032.00,15032.00
mean,19982.63,0.21
std,5742.70,0.24
min,3458.00,-10.92
25%,16650.50,0.05
50%,20401.00,0.21
75%,24690.00,0.32
max,29874.00,4.24


**__Catatan__** : <p>
Nilai positif menunjukkan peningkatan berat dari sampling sebelumnnya, sedangkan nilai minus menunjukkan penurunan berat dari sampling sebelumnya

# Data Preparation for Model

- Survival Rate - End

In [18]:
# Aggregate & Join All Features

#cycle
cycle_all = result = SR.groupby('cycle_id').agg({
    'pond_length': lambda x: x.unique().tolist()[0],
    'pond_width': lambda x: x.unique().tolist()[0],
    'pond_depth': lambda x: x.unique().tolist()[0],
    'long_cycle(day)': lambda x: x.unique().tolist()[0]
}).reset_index().rename(columns={'pond_length':'avg_pond_length','pond_width':'avg_pond_width','pond_depth':'avg_pond_depth'})

#feed 
feed_all = feeds.groupby('cycle_id')[['quantity']].sum().rename(columns={'quantity':'total_feed(kg)'}).reset_index()

#feed tray
df_ft = feed_tray.groupby('cycle_id')[['tray_number','feed_remaining_percent']].mean().reset_index().rename(columns={'tray_number':'avg_tray_number', 'feed_remaining_percent':'avg_feed_remain%'})
df_ft['avg_tray_number'] = round(df_ft['avg_tray_number'],0)

#fasting 
fasting_join = pd.concat([fasting, fastings], ignore_index=True)
fasting_all = fasting_join.groupby('cycle_id')[['fasting']].count().reset_index().sort_values(by='cycle_id').rename(columns={'fasting':'count_fasting'})

#mortal
mortal_all = mortal.groupby('cycle_id')[['quantity']].sum().reset_index().rename(columns={'quantity':'total_mortal'})
mortal_all['avg_weight_mortal'] = mortal.groupby('cycle_id')[['average_weight']].sum().reset_index().average_weight.tolist()

#samplings
samp_all = sampling.groupby('cycle_id')[['average_weight']].mean().reset_index().rename(columns={'average_weight':'avg_weight_sample'})


In [19]:
measure.isnull().sum()

pond_id                     0
cycle_id                    0
measured_date               0
morning_temperature     41855
evening_temperature     49839
morning_do              61324
evening_do              65098
morning_salinity        46388
evening_salinity        73957
morning_pH              20115
evening_pH              78101
transparency            61750
turbidity              138884
ammonia                135302
nitrate                136888
nitrite                129293
alkalinity             125533
hardness               133525
calcium                132458
magnesium              132430
carbonate              133762
bicarbonate            132372
tom                    131774
total_plankton_        126429
dtype: int64

**__Catatan__** : <p>
Tabel measure diasumsikan sulit untuk dimasukan sebagai variabel karena data terlalu banyak null dan tidak dapat dihandling fillna(0) karena diasumsikan data tersebut bersifat real time atau tidak mungkin nan

In [20]:
list_tabel = [cycle_all, feed_all, df_ft, fasting_all, mortal_all,samp_all]

def join(data):
    for df in list_tabel :
        data = pd.merge(data, df, how='left', on='cycle_id')
        data.fillna(0,inplace=True)
    return data

In [21]:
sr_model = df_sr.copy()
sr_model = join(sr_model)

sr_model.head(3)

,cycle_id,total_harvest(kg),total_seed,survival_rate(%),avg_pond_length,avg_pond_width,avg_pond_depth,long_cycle(day),total_feed(kg),avg_tray_number,avg_feed_remain%,count_fasting,total_mortal,avg_weight_mortal,avg_weight_sample
0,3458,444548.02,566669,78.45,80.09,60.07,1.20,112,9987.00,0.00,0.00,0.00,0.00,0.00,13.59
1,3459,440387.88,566669,77.72,78.62,58.97,1.20,121,10913.00,0.00,0.00,0.00,0.00,0.00,14.32
2,4036,154350.00,172250,89.61,39.50,18.40,1.10,39,480.10,0.00,0.00,1.00,0.00,0.00,0.00


In [22]:
# save dataset
sr_model.to_csv('Dataset/sr_model.csv')

- Average Body Weight

In [23]:
adg_model = adg.copy()
adg_model = join(adg_model)

adg_model.head(3)

,cycle_id,sampled_at,average_weight,distance_date,average_daily_gain,avg_pond_length,avg_pond_width,avg_pond_depth,long_cycle(day),total_feed(kg),avg_tray_number,avg_feed_remain%,count_fasting,total_mortal,avg_weight_mortal,avg_weight_sample
0,3458.00,2020-04-10,4.37,0,0.00,80.09,60.07,1.20,112.00,9987.00,0.00,0.00,0.00,0.00,0.00,13.59
1,3458.00,2020-04-17,5.23,7,0.12,80.09,60.07,1.20,112.00,9987.00,0.00,0.00,0.00,0.00,0.00,13.59
2,3458.00,2020-04-24,6.69,7,0.21,80.09,60.07,1.20,112.00,9987.00,0.00,0.00,0.00,0.00,0.00,13.59


In [24]:
# save dataset
abw_model.to_csv('Dataset/abw_model.csv')

- Biomass

In [25]:
bio = df_sr_day.copy()
bio['average_weight(gr)'] = (bio['weight']*1000) / bio['total_harvest(kg)']
bio['est_remaining_population'] = bio['total_seed'] - bio['total_harvest_acc(kg)']

bio['biomass(kg)'] = (bio['average_weight(gr)'] * bio['est_remaining_population'])/1000

In [26]:
# Result
bio[bio['biomass(kg)']> 0][['cycle_id','biomass(kg)']].describe()

,cycle_id,biomass(kg)
count,7048.00,7048.00
mean,19860.86,2105.01
std,5678.75,3836.70
min,3458.00,0.00
25%,16344.00,548.25
50%,19688.00,1323.73
75%,24691.00,2608.16
max,29874.00,97871.00


In [27]:
#Anomali
bio[bio['biomass(kg)']< 0].describe()

,cycle_id,weight,size,total_harvest(kg),total_harvest_acc(kg),total_seed,survival_rate(%),average_weight(gr),est_remaining_population,biomass(kg)
count,959.00,959.00,959.00,959.00,959.00,959.00,959.00,959.00,959.00,959.00
mean,21100.06,1759.41,120.25,157067.37,508736.82,203689.31,1073.58,14.83,-305047.50,-2871.37
std,5337.97,11811.38,415.58,1494505.70,1977384.68,142356.45,7531.06,8.75,1967087.91,16236.91
min,4038.00,0.50,19.00,37.30,3127.00,10.00,100.01,0.10,-39950000.00,-249687.50
25%,18675.00,61.80,48.40,5604.70,125117.00,100000.00,107.11,8.00,-100500.00,-1340.11
50%,21596.00,350.00,80.00,30000.00,261209.18,175000.00,117.33,12.50,-24500.00,-324.33
75%,24965.50,1270.50,125.00,97807.50,367283.07,286607.00,153.06,20.66,-8730.25,-97.34
max,29679.00,250000.00,10000.00,40000000.00,40000000.00,1200000.00,115817.39,52.63,-16.00,-0.15


**__Catatan__** : <p>
Berdasarkan analisa, terdapat banyak siklus yang memiliki total_seed dan harvest yang tidak sesuai, seperti total harvest > total_seed, sehingga menghasilkan biomassa(%) dengan nilai minus

In [28]:
bio_model = bio.drop(['total_harvest_acc(kg)'],axis=1)
bio_model = join(bio_model)

bio_model.to_csv('Dataset/bio_model.csv')

- Revenue

In [29]:
price = harvest[harvest['selling_price'].notna()].sort_values(by='cycle_id')[['cycle_id','size','weight','selling_price']].sort_values(by='cycle_id').reset_index(drop='first')

price.to_csv('Dataset/price.csv')